In [6]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.model_selection import KFold, cross_val_score

from tensorflow import random

In [2]:
colnames = ['CIC0', 'SM1_Dz(Z)', 'GATS1i', 'NdsCH', 'NdssC','MLOGP', 'LC50']
data = pd.read_csv('datasets/qsar_fish_toxicity.csv', sep=';', names=colnames)

In [3]:
X = data.drop('LC50', axis=1)
y = data['LC50']

In [5]:
def build_model_1():
    model = Sequential()
    
    model.add(Dense(4, activation='relu', input_dim=X.shape[1]))
    model.add(Dense(1))
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    return model

def build_model_2():
    model = Sequential()
    
    model.add(Dense(8, activation='relu', input_dim=X.shape[1]))
    model.add(Dense(1))
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    return model

def build_model_3():
    model = Sequential()
    
    model.add(Dense(4, activation='relu', input_dim=X.shape[1]))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1))
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    return model

In [7]:
seed = 1
np.random.seed(seed)
random.set_seed(seed)

In [8]:
results_1 = []

models = [build_model_1, build_model_2, build_model_3]

for model in range(len(models)):
    model = KerasRegressor(build_fn=models[model], batch_size=20, epochs=100, verbose=0, shuffle=False)
    
    kf = KFold(n_splits=3)
    result = cross_val_score(model, X, y, cv=kf)
    
    results_1.append(result)

In [9]:
print("Cross-Validation Loss do Modelo 1 =", abs(results_1[0].mean()))
print("Cross-Validation Loss do Modelo 2 =", abs(results_1[1].mean()))
print("Cross-Validation Loss do Modelo 3 =", abs(results_1[2].mean()))

Cross-Validation Loss do Modelo 1 = 1.030479402851683
Cross-Validation Loss do Modelo 2 = 0.9583349300839368
Cross-Validation Loss do Modelo 3 = 0.9740844359134707


In [10]:
np.random.seed(seed)
random.set_seed(seed)

results_2 = []
epochs = [100, 150]
batches = [15, 20]

for e in range(len(epochs)):
    for b in range(len(batches)):
        model = KerasRegressor(build_fn=build_model_2, batch_size=batches[b], epochs=epochs[e], verbose=0, shuffle=False)
        
        kf = KFold(n_splits=3)
        result = cross_val_score(model, X, y, cv=kf)
        results_2.append(result)

In [22]:
c = 0

for e in range(len(epochs)):
    for b in range(len(batches)):
        print('Batch_size =', batches[b],', ' 'Epochs =', epochs[e],', ' 'Loss no teste = ', abs(results_2[c].mean()))
        c += 1

Batch_size = 15 , Epochs = 100 , Loss no teste =  0.9382871842186092
Batch_size = 20 , Epochs = 100 , Loss no teste =  0.9582501313525756
Batch_size = 15 , Epochs = 150 , Loss no teste =  0.8939150340429367
Batch_size = 20 , Epochs = 150 , Loss no teste =  0.9527814686270144


In [23]:
def build_model_2(activation='relu', optimizer='adam'):
    model = Sequential()
    
    model.add(Dense(8, activation=activation, input_dim=X.shape[1]))
    model.add(Dense(1))
    
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    return model

In [24]:
results_3 = []
activations = ['relu', 'tanh']
optimizers = ['sgd', 'adam', 'rmsprop']

np.random.seed(seed)
random.set_seed(seed)

for o in range(len(optimizers)):
    for a in range(len(activations)):
        optimizer = optimizers[o]
        activation = activations[a]
        
        model = KerasRegressor(build_fn=build_model_2, epochs = 100, batch_size=20, verbose=0, shuffle=False)
        
        kf = KFold(n_splits=3)
        
        result = cross_val_score(model, X, y, cv=kf)
        results_3.append(result)

In [27]:
c = 0

for o in range(len(optimizers)):
    for a in range(len(activations)):
        print('Activation =', activations[a],',', 'Optimizer =', optimizers[o], ',', 'Loss no test =', abs(results_3[c].mean()))
        c += 1

Activation = relu , Optimizer = sgd , Loss no test = 0.9572524036535927
Activation = tanh , Optimizer = sgd , Loss no test = 0.9582640252745939
Activation = relu , Optimizer = adam , Loss no test = 0.9006734350314405
Activation = tanh , Optimizer = adam , Loss no test = 0.9858099008546614
Activation = relu , Optimizer = rmsprop , Loss no test = 0.9365009631216058
Activation = tanh , Optimizer = rmsprop , Loss no test = 0.9717393287848438
